# ntakt Functions
ImgLib2 offers an API to generate virtual functions through the `FunctionRealRandomAccessible` that are evaluated at real locations on demand.

In [1]:
// set up dependencies
// use local maven repository; not yet deployed to remote maven repositories.
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")

// uncomment to search in your local maven repo
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
// @file:DependsOn("org.ntakt:ntakt:0.1.0-SNAPSHOT")

// uncomment to search in jitpack
@file:DependsOn("org.ntakt:ntakt:main-SNAPSHOT")

@file:DependsOn("net.imglib2:imglib2-label-multisets:0.8.1")
@file:DependsOn("sc.fiji:bigdataviewer-vistools:1.0.0-beta-21")
%use lets-plot

In [2]:
// for hsv to rgb
import java.awt.Color

import bdv.util.BdvOptions
import kotlin.math.abs
import kotlin.math.cos
import kotlin.math.min
import kotlin.math.pow
import kotlin.math.sin
import kotlin.math.sqrt
import kotlin.random.Random
import net.imglib2.KDTree
import net.imglib2.RealPoint
import net.imglib2.converter.Converter
import net.imglib2.interpolation.neighborsearch.InverseDistanceWeightingInterpolatorFactory
import net.imglib2.neighborsearch.KNearestNeighborSearchOnKDTree
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.integer.IntType
import net.imglib2.view.Views
import org.ntakt.*

In [3]:
val euclideanNormSquared2D = ntakt.function(2, { ntakt.types.double }) { s, t ->
    t.setReal(s.getDoublePosition(0).pow(2) + s.getDoublePosition(1).pow(2))
}
val interval1 = intArrayOf(-256, -256, 0, 0).intervalMinMax
val interval2 = intArrayOf(0, 0, +256, +256).intervalMinMax
val bdv = euclideanNormSquared2D.rastered[interval1].show("Euclidean norm 2D 1", BdvOptions.options().is2D())
euclideanNormSquared2D.rastered[interval2].show("Euclidean norm 2D 2", bdv=bdv)

bdv.util.BdvStackSource@5b5caf08

## Fractals

In [4]:
// utility functions to convert into ARGB space, if desired
// modify this for cool looking results
inline fun makeConverter(
    n: Int, 
    saturation: (Double) -> Double = { 0.3 }, 
    value: (Double) -> Double = { it },
    hue: (Double) -> Double): Converter<IntType, ARGBType>
    {
    val colors = IntArray(n) {
        val fraction = it.toDouble() / n
        val h = hue(fraction)
        val sat = saturation(fraction)
        val v = value(fraction)
        val rgb = Color.HSBtoRGB(h.toFloat(), sat.toFloat(), v.toFloat())
        val r = (rgb shr 16) and 0xff
        val g = (rgb shr  8) and 0xff
        val b = (rgb shr  0) and 0xff
        val argb = ARGBType.rgba(r, g, b, 0)
        if (it == n - 1) 0 else argb
    }
    return Converter { s, t -> t.set(colors[min(s.integer, n - 1)]) }
}

fun makeRandomConverter(n: Int, random: Random) = makeConverter(n) { random.nextDouble() }
fun makeGrayscaleConverter(n: Int) = makeConverter(n, { 0.0 }, { it }) { 0.0 }
fun makeMulticolorConverter(
    n: Int, 
    saturation: (Double) -> Double = { 0.3 }, 
    value: (Double) -> Double = { it },
    vararg fractionToHue: Pair<Double, Double>): Converter<IntType, ARGBType>
{
    require(fractionToHue.isNotEmpty())
    val hues = fractionToHue.map { ntakt.types.double.createWithValue(it.second) }
    val points = fractionToHue.map { RealPoint(it.first) }
    val nnSearch = KNearestNeighborSearchOnKDTree(KDTree(hues, points), 2)
    val interpolated = Views.interpolate(nnSearch, InverseDistanceWeightingInterpolatorFactory())
    val rra = interpolated.realRandomAccess()
    return makeConverter(n, saturation, value) { rra.setPosition(it, 0); rra.get().realDouble }
}
fun makeTwoColorConverter(
    n: Int,
    hueLow: Double = 180.0 / 360.0,
    hueHigh: Double = 60.0 / 360.0,
    saturation: Double = 0.3,
    value: (Double) -> Double = { 1.0 }) 
        = makeConverter(
            n,
            { saturation * 2.0 * abs(it-0.5) },
            value,
            { (if (it < 0.5) hueLow else hueHigh) })

### Julia

Wikipedia has a list of cool values for c: https://en.wikipedia.org/wiki/Julia_set#Quadratic_polynomials

In [5]:
// c&p from Stephan Saalfeld's Java code snippet
// Julia with c = 0.2 + i0.6
fun julia(real: Double, imaginary: Double, r: Double, cRe: Double, cIm: Double, iterations: Int): Int {
    var i = 0
    var re = real
    var im = imaginary
    while (i < iterations && re * re + im * im < r*r) {
        val e = re * re - im * im
        im = 2 * re * im + cIm
        re = e + cRe
        ++i
    }
    return i
}

#### Fixed c and radius

In [6]:
val iterations = 256
val cIm = 0.6
val cRe = 0.2
// play around with r; for r ~ 1, very "interesting looking results"
val r = 2.0
val fractal = ntakt.function(2, { ntakt.types.int }) { s, t ->
    t.setInteger(julia(s.getDoublePosition(0), s.getDoublePosition(1), r, cRe, cIm, iterations))
}
val interval = intArrayOf(-1, -1, +1, +1).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeRandomConverter(iterations, Random(1)))
// show either convertedFractal or fractal
convertedFractal.show("julia", interval, BdvOptions.options().is2D()).setDisplayRange(0.0, 64.0)

#### Fixed c and variable radius

In [7]:
val iterations = 256
val cIm = 0.6
val cRe = -0.4
val fractal = ntakt.function(3, { ntakt.types.int }) { s, t ->
    t.setInteger(julia(s.getDoublePosition(0), s.getDoublePosition(1), s.getDoublePosition(2), cRe, cIm, iterations))
}
val interval = intArrayOf(-1, -1, 0, +1, +1, +4).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeRandomConverter(iterations, Random(1)))
// show either convertedFractal or fractal
convertedFractal.show("julia", interval, BdvOptions.options()).setDisplayRange(0.0, 64.0)

#### Fixed radius and variable c 

In [8]:
val iterations = 256
val magnitude = 0.7885 // modify this
val r = 2.0
val fractal = ntakt.function(3, { ntakt.types.int }) { s, t ->
    val angleInRadians = s.getDoublePosition(2)
    val cRe = magnitude * cos(angleInRadians)
    val cIm = magnitude * sin(angleInRadians)
    t.setInteger(julia(s.getDoublePosition(0), s.getDoublePosition(1), r, cRe, cIm, iterations))
}
val interval = intArrayOf(-1, -1, 0, +1, +1, +6).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeRandomConverter(iterations, Random(1)))
// show either convertedFractal or fractal
convertedFractal.show("julia", interval, BdvOptions.options()).setDisplayRange(0.0, 64.0)

### Mandelbrot

In [9]:
fun mandelbrot(real: Double, imaginary: Double, r: Double, iterations: Int): Int {
    var i = 0
    var re = 0.0
    var im = 0.0
    while (i < iterations && re * re + im * im < r*r) {
        val e = re * re - im * im + real
        im = 2 * re * im + imaginary
        re = e
        ++i
    }
    return i
}

In [10]:
val iterations = 256
val fractal = ntakt.function(3, { ntakt.types.int }) { s, t ->
    t.setInteger(mandelbrot(s.getDoublePosition(0), s.getDoublePosition(1), s.getDoublePosition(2), iterations))
}
val interval = intArrayOf(-1, -1, 0, +1, +1, +4).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeRandomConverter(iterations, Random(1)))
// show either convertedFractal or fractal
convertedFractal.show("mandelbrot", interval, BdvOptions.options()).setDisplayRange(0.0, iterations.toDouble())

### Tricorn

In [11]:
fun tricorn(real: Double, imaginary: Double, r: Double, iterations: Int): Int {
    var i = 0
    var re = 0.0
    var im = 0.0
    while (i < iterations && re * re + im * im < r*r) {
        val e = re * re - im * im + real
        im = -2 * re * im + imaginary
        re = e
        ++i
    }
    return i
}

In [12]:
val iterations = 256
val fractal = ntakt.function(3, { ntakt.types.int }) { s, t ->
    t.setInteger(tricorn(s.getDoublePosition(0), s.getDoublePosition(1), s.getDoublePosition(2), iterations))
}
val interval = intArrayOf(-1, -1, 0, +1, +1, +4).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeRandomConverter(iterations, Random(1)))
// show either convertedFractal or fractal
convertedFractal.show("tricorn", interval, BdvOptions.options()).setDisplayRange(0.0, iterations.toDouble())